In [3]:
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis as PA
from modlamp.descriptors import PeptideDescriptor, GlobalDescriptor
from sklearn.model_selection import train_test_split,KFold
import pandas as pd
import os, re, math, platform
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from numpy import linalg as la
import os
import random
import warnings
warnings.filterwarnings('ignore')
import random
from fea_extract import read_fasta,insert_AAC,insert_DPC,insert_CKSAAGP,insert_CTD,insert_PAAC,insert_AAI,insert_GTPC,insert_QSO,insert_AAE,insert_PSAAC,insert_word2int,insert_ASDC
from tools import del_data
seed = 10
from pathlib import Path
Path('./results/process_data/').mkdir(exist_ok=True,parents=True)
Path('./results/balance/').mkdir(exist_ok=True,parents=True)

In [5]:
# 定义函数
def del_data(inFile):
    seq = read_fasta(inFile)
    seqname = seq.to_numpy()
    newseq = []
    j = seqname.shape[0]
    for i in range(j):
        if 6 <= len(seqname[i][1])<=100:
            newseq.append(seqname[i])
    newseq = np.array(newseq)
    print('序列去除小于6序列后的维度：', newseq.shape)
    newseq = pd.DataFrame(data=newseq, columns=["Id", "Sequence"])
    return newseq

In [6]:
seq_Anti_cov = del_data('data/ANTI-COV.csv')
seq_AVP = del_data('data/Anti-Virus.faa')

序列去除小于6序列后的维度： (157, 2)
序列去除小于6序列后的维度： (1984, 2)


In [6]:
from tools import pro_data

In [7]:
#划分训练集和测试集80%，20%
prefix = [seq_AVP,seq_Anti_cov]
Prefix = ['Seq_avp','Seq_anticov']
for i in range(2):
    df_train,df_test = train_test_split(prefix[i],random_state=seed,test_size=.2)
    df_train.to_csv(os.path.join( "data/train/{:s}_train.csv".format(Prefix[i])), index=False)
    df_test.to_csv(os.path.join("data/test/{:s}_test.csv".format(Prefix[i])), index=False)
    print("Done!")

Done!
Done!


In [9]:
#分别读取不同样本的训练集和测试集
train_sets = {
    lab:pd.read_csv('data/train/{:s}_train.csv'.format(lab))
    for lab in ['Seq_avp','Seq_anticov']
}
test_sets = {
    lab:pd.read_csv('data/test/{:s}_test.csv'.format(lab))
    for lab in ['Seq_avp','Seq_anticov']
}
train_sets['Seq_anticov'].loc[:, 'Label'] = 1 #loc[]通过行标签索引行数据
train_sets['Seq_avp'].loc[:, 'Label'] = 0 #负数据集
test_sets['Seq_anticov'].loc[:, 'Label'] = 1
test_sets['Seq_avp'].loc[:, 'Label'] = 0

#对序列进行随机打乱
all_train = pd.concat([train_sets['Seq_anticov'],train_sets['Seq_avp']],ignore_index='ignore')
all_test = pd.concat([test_sets['Seq_anticov'],test_sets['Seq_avp']],ignore_index='ignore')
X_train = all_train.iloc[:,0:2]
X_test = all_test.iloc[:,0:2]
y_train = all_train['Label']
y_test = all_test["Label"]
X_test.to_csv('data/test/X_test.csv',index = False)
y_test.to_csv('data/test/y_test.csv',index = False)
X_train.to_csv('data/train/X_train.csv',index = False)
y_train.to_csv('data/train/y_train.csv',index = False)

###### 对训练样本进行编码

In [11]:
insert_list=[insert_AAC,insert_DPC,insert_CKSAAGP,insert_PSAAC,insert_CTD,insert_GTPC,
             insert_QSO,insert_AAE,insert_AAI,insert_ASDC,insert_PAAC,pro_data]
insert_str=["insert_AAC","insert_DPC","insert_CKSAAGP","insert_PSAAC","insert_CTD",
            "insert_GTPC","insert_QSO","insert_AAE","insert_AAI","insert_ASDC","insert_PAAC",'insert_All_data']

In [12]:
n = len(insert_list)
for j in range(n):
    print("encoding{}_{}".format(str(j),insert_str[j][7:]))
    X_test = pd.read_csv('data/test/X_test.csv')
    X_train = pd.read_csv('data/train/X_train.csv')
    df_seq_train = insert_list[j](X_train)
    df_seq_train.to_csv('results/process_data/train_{}.csv'.format(insert_str[j][7:]),index =False )
    df_seq_test = insert_list[j](X_test)
    df_seq_test.to_csv('results/process_data/test_{}.csv'.format(insert_str[j][7:]),index =False )

encoding0_AAC
encoding1_DPC
encoding2_CKSAAGP
encoding3_PSAAC
encoding4_CTD
encoding5_GTPC
encoding6_QSO
encoding7_AAE
encoding8_AAI
encoding9_ASDC
encoding10_PAAC
encoding11_All_data


###### 对于Word2int编码

In [7]:
from tools import padseq

In [8]:
X_test = pd.read_csv('data/test/X_test.csv')
X_train = pd.read_csv('data/train/X_train.csv')
new_seq_train = padseq(X_train)
new_seq_test = padseq(X_test)
word2_seq_train = insert_word2int(new_seq_train)
word2_seq_train.to_csv('results/process_data/train_word2int.csv',index=False)

word2_seq_test = insert_word2int(new_seq_test)
word2_seq_test.to_csv('results/process_data/test_word2int.csv',index=False)

######  对平衡数据集进行编码

In [15]:
insert_list=[insert_AAC,insert_DPC,insert_CKSAAGP,insert_PSAAC,insert_CTD,insert_GTPC,
             insert_QSO,insert_AAE,insert_AAI,insert_ASDC,insert_PAAC]
insert_str=["insert_AAC","insert_DPC","insert_CKSAAGP","insert_PSAAC","insert_CTD",
            "insert_GTPC","insert_QSO","insert_AAE","insert_AAI","insert_ASDC","insert_PAAC"]

In [16]:
n = len(insert_list)
for j in range(n):
    print("encoding{}_{}".format(str(j),insert_str[j][7:]))
    X_test = pd.read_csv('data/test/X_test.csv')
    X_train = pd.read_csv('data/processdata/blearn_train.csv')
    df_seq_train = insert_list[j](X_train)
    df_seq_train.to_csv('results/balance/train_{}.csv'.format(insert_str[j][7:]),index =False )
    df_seq_test = insert_list[j](X_test)
    df_seq_test.to_csv('results/balance/test_{}.csv'.format(insert_str[j][7:]),index =False )

encoding0_AAC
encoding1_DPC
encoding2_CKSAAGP
encoding3_PSAAC
encoding4_CTD
encoding5_GTPC
encoding6_QSO
encoding7_AAE
encoding8_AAI
encoding9_ASDC
encoding10_PAAC


In [17]:
from tools import padseq

In [18]:
X_test = pd.read_csv('data/test/X_test.csv')
X_train = pd.read_csv('data/processdata/blearn_train.csv')
new_seq_train = padseq(X_train)
new_seq_test = padseq(X_test)
word2_seq_train = insert_word2int(new_seq_train)
word2_seq_train.to_csv('results/balance/train_word2int.csv',index=False)

word2_seq_test = insert_word2int(new_seq_test)
word2_seq_test.to_csv('results/balance/test_word2int.csv',index=False)

In [16]:
import pandas as pd
import numpy as np

In [17]:
X_test = pd.read_csv('data/test/X_test.csv').to_numpy()
X_test.shape

(429, 2)

In [18]:
x_TEST=[]
for i in range(429):
    x_TEST.append('>'+X_test[i][0])
    x_TEST.append(X_test[i][1])

In [20]:
x_test = pd.DataFrame(x_TEST)
x_test.to_csv('x_Test.csv',index=False)